# Casadi with IpOpt: howto by example

In this notebook we give the basic commands to build a numerical program formulated with Casadi and solved with IpOpt. The example is to solve a very simple toy problem involving a rotation
$$
min_w   || R p - p' ||^2  
$$
with $R:= exp(w_\times)$, $p$ and $p'$ are two known 3d vectors.

The result is displayed in Meshcat.

## Setup 
We will need casadi, numpy, pinocchio for simple SO3 algebra and meshcat for renderig. 
If you dont have casadi, install it with `sudo apt install robotpkg-py38-casadi`.

In [ ]:
import casadi
import pinocchio as pin
from casadi_so3 import exp3,log3
from pinocchio.utils import rotate
import numpy as np
import time
from utils.meshcat_viewer_wrapper import MeshcatVisualizer

## Reference values
We define an arbitrary reference trajectory for +pdes+ which is rotating and oscillating around the surface of a sphere.

In [ ]:
p = np.array([0.1, 0.2, 0.3])
omega = 2*np.pi*.5
pdes = [ rotate('x',t)@rotate('y',2*t-5)@ ((1+.2*np.sin(t*omega))*p)
         for t in np.arange(0,5,.02) ]
T = len(pdes)

## Problem formulation
The problem is handled by a +casadi.Opti+ object, which enables to define Casadi variables and expression graphs with them. Let's formulate the problem of the header by this mean. 


In [ ]:
# Create the casadi optimization problem
opti = casadi.Opti()

The variables are a collection of SO(3) along a temporal line, defined by their angle-vector representation $[w_0...w_{T-1}]$. We accordingly define the sequene of rotation matrix $[R_0...R_{T-1}]$. You can see them as variables, but they are actually expression graphs built from the $w_t$.

In [ ]:
# The optimization variable is the angle-vector w and the associated rotation R=exp(w)
ws = [ opti.variable(3) for t in range(T) ]
Rs = [ exp3(w) for w in ws ]

We now build the expression graph for the cost. The mathematical operations are gathered in a function to be clean.

In [ ]:
def make_a_cost_expression(p, pdes, T, ws, Rs):
    totalcost = 0

    # Beware: casadi matrix product is @ like numpy array product
    for t in range(T):
        totalcost += 0.5 * casadi.sumsqr(Rs[t] @ p - pdes[t])
        if t>0:
            totalcost += 0.5 * casadi.sumsqr( log3(Rs[t-1].T @ Rs[t]) )
            #totalcost += 0.5 * casadi.sumsqr( ws[t] - ws[t-1])
    return totalcost

In [ ]:
totalcost = make_a_cost_expression(p, pdes, T, ws, Rs) 

+totalcost+ is an expression, made from the variables $w_t$. This expression can be used by Casadi to evaluate the cost (given candidate values for the $w_t$, give me the value of the cost) but can also be algorithmically differentiated to obtain gradient or Hessian.
We specify to Casadi what is the expression to minimize.

In [ ]:
opti.minimize(totalcost)

We can now ask Casadi to call IpOpt to solve it.

## Solve
Casadi will call an external solver to optimize the given problem. We are going to use +IpOpt+, which is not the best solver for the simple unconstrained sparse problem we are proposing, but it is convenient and strong, so ... why not?

In [ ]:
opti.solver("ipopt")

Then simply solve:

In [ ]:
sol = opti.solve()

### Warn start
The decision variables can be initialized to accelerate the search.

In [ ]:
for t in range(T):
    opti.set_initial(ws[t],np.array([.1,.1,.1]))

In [ ]:
sol = opti.solve()

### Silence 
The solver can be given some extra options, for example here to be silent:

In [ ]:
opts = {'ipopt.print_level': 0, 'print_time': 0, 'ipopt.sb': 'yes'}
opti.solver("ipopt",opts)
sol = opti.solve()

## Recovering the optimal values

Use +opti.value(...)+ to get the value of any expresion you like.
For example here, the value of the decision variable at the optimum and the corresponding rotation matrices are stored in 2 arrays.

In [ ]:
ws_sol = [ opti.value(w) for w in ws ]
Rs_sol = [ opti.value(R) for R in Rs ]

Sanity check:

In [ ]:
for R_sol in Rs_sol:
    assert np.allclose(R_sol @ R_sol.T, np.eye(3))

## In case IpOpt does not converge
Then it raises an error. The candidate values are then not directly available but can be recovered.

In [ ]:
try:
    sol = opti.solve_limited()
    ws_sol = [ opti.value(w) for w in ws ]
    Rs_sol = [ opti.value(R) for R in Rs ]
except:
    print('ERROR in convergence, plotting debug info.')
    ws_sol = [ opti.debug.value(w) for w in ws ]
    Rs_sol = [ opti.debug.value(R) for R in Rs ]

## Display

In [ ]:
viz = MeshcatVisualizer()
viz.viewer.jupyter_cell()

In [ ]:
pointID = "world/point"
viz.addSphere(pointID, 0.1, [1, 0, 0, 1])
pointdesID = "world/pointdes"
viz.addSphere(pointdesID, 0.1, [0, 1, 0, 1])
boxID = "world/box"
viz.addBox(boxID, (p * 2).tolist(), [1, 1, 0, .1])

def viewtraj():
    for t,[R_sol,pt] in enumerate(zip(Rs_sol,pdes)):
        viz.applyConfiguration(pointdesID, pt.tolist() + [0, 0, 0, 1])
        viz.applyConfiguration(pointID, (R_sol @ p).tolist() + [0, 0, 0, 1])
        viz.applyConfiguration(boxID, [0, 0, 0] + pin.Quaternion(R_sol).coeffs().tolist())
        time.sleep(1e-2)

In [ ]:
viewtraj()